In [2]:
import os
import gzip
import time
import pandas as pd
import urllib.request
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import display

In [3]:
##########################################
### MODIFY THIS, to store the DataSet: ###
##########################################

localDataUrl = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSets'

##########################################
##########################################

url = 'http://insideairbnb.com/get-the-data.html'

response = get(url);
soup = BeautifulSoup(response.text)

# Define scraping functions

In [1]:
# beautiful tutorial: https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/

def url_to_df(soup):
    for d in soup.find_all(class_='archived'): 
        d.decompose()

    url_content = pd.DataFrame()
    col_names = []

    for table in soup.find_all('table'):

        # find table elements 
        for row in table.find_all('tr'):

            # find columns names:
            th_tags = row.find_all('th') 

            # if not empty
            if len(th_tags) > 0:
                if len(col_names) == 0:
                    for t in th_tags:
                        col_names.append(t.text)

            # find rows contents
            td_tags = row.find_all('td')

            content = []
            for column in td_tags:
                try:
                    for el in column:
                        content.append(el.get('href'))
                    content.append(column.text)
                except Exception:
                    content.append(column.text)
                    # pass

            # check if list is not empty
            if content:
                url_content = url_content.append(pd.Series(content), ignore_index = True)

    col_names = ['Date Compiled', 'Country/City', 'URL', 'File Name', 'Description']
    url_content.columns = col_names

    # format date
    url_content['Date Compiled'] = pd.to_datetime(url_content['Date Compiled'], errors='coerce')
    url_content['Date Compiled'] = url_content['Date Compiled'].astype('str')

    display(url_content.head())
    
    return url_content


def download_website(url_content):
    for index, row in url_content.iterrows():

        file_name = localDataUrl + '/' + row['Date Compiled'] + '_' + row['Country/City'] + '_' + row['File Name']
        print('Downloading... ' + str(index+1) +'/' + str(url_content.shape[0]) + ' : ' + file_name)

        try:
            [fileName, header] = urllib.request.urlretrieve(row['URL'], file_name)

            # decompress gzip file
            if fileName[-2:]=='gz':
                input = gzip.GzipFile(fileName, 'rb')
                s = input.read()
                input.close()
                # delete gzip file
                os.remove(fileName)

                savingName = fileName[0:-3]
                if 'listings' in fileName:
                    savingName = savingName[0:-4] + '_detailed.csv'
                output = open(savingName, 'wb')
                output.write(s)
                output.close()

        except Exception:
            print('Download failed')
    
        # pause code for a second so that we are not spamming the website with requests. 
        # This helps to avoid getting flagged as a spammer
        time.sleep(1)
    



# Scrape and download the dataset

In [ ]:
url_content = url_to_soup(soup)
download_website(url_content)

In [ ]:
'''
# remove all the archived classes
for d in soup.find_all(class_='archived'): 
    d.decompose()
file_classes = soup.find_all(class_ = lambda value: value and value.startswith('table table'))

for city in file_classes:
    files_links = city.find_all('a', class_=None)
    for link in files_links:
        file_url = link.get('href')
        # print(file_url)
        
r0 = file_classes[0].find_all('td', class_='')
r0
date = r0[0].text
city = r0[1].text
link = r0[2]
file_name = r0[3].text
test = r0[4].text
t0 = link.find_all(href = True)
for i in link:
    print(i.get('href'))
'''